<a href="https://colab.research.google.com/github/MuroriM/GEM-table-to-text-through-AMR/blob/main/AMR_valid_postprocessor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AMR linearizer  
This converts AMRs from Penman notation to their linear equivalent. This includes stripping of variables, newline characters, and other whitespaces.

In [ ]:
!pip install amrlib

In [ ]:
# remove files from previous sessions

!ls
!rm -r AMR test_web smatch
!ls

AMR  sample_data  test_web
rm: cannot remove 'smatch': No such file or directory
sample_data


In [ ]:
# load dev, train, test, webnlg data

# /content/test_web/webnlg_dev_parsed.jsonl.tf
# /content/test_web/webnlg_test_parsed.jsonl.tf
# /content/test_web/webnlg_test_parsed.jsonl.tf
# respective paths

!git clone https://github.com/MuroriM/test_web.git
# https://github.com/MuroriM/test_web

Cloning into 'test_web'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 16 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (16/16), done.


# Character-based AMR translation

In [ ]:
!git clone https://github.com/RikVN/AMR.git
# https://github.com/RikVN/AMR

Cloning into 'AMR'...
remote: Enumerating objects: 127, done.
remote: Total 127 (delta 0), reused 0 (delta 0), pack-reused 127
Receiving objects: 100% (127/127), 209.88 KiB | 2.12 MiB/s, done.
Resolving deltas: 100% (54/54), done.


In [ ]:
%cd /content/AMR

/content/AMR


In [ ]:
!git clone https://github.com/snowblink14/smatch

Cloning into 'smatch'...
remote: Enumerating objects: 147, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 147 (delta 0), reused 2 (delta 0), pack-reused 144
Receiving objects: 100% (147/147), 74.56 KiB | 1.52 MiB/s, done.
Resolving deltas: 100% (79/79), done.


In [ ]:
!pip install -r requirements.txt

In [ ]:
# This step is important! 
# Though a temporary fix
!cp amr_utils.py restoreAMR
!cp best_amr_permutation.py restoreAMR
!cp var_free_amrs.py restoreAMR

In [ ]:
## Create test dataset
!head -3 /content/test_web/webnlg_test_target.json > /content/test_web/test_repo.jsonl

In [ ]:
#@title
# # Converts linearized AMRs to Penman notation
# # Convert original WebNLP Source pairs
# # Not needed for our current use

# import json

# # Loads linear AMRs from dataset and stores the Penman AMRs in new dataset
# def linear_to_penman_dataset(input_file):
  
#     modified_entries = []
#     for line in open(input_file, 'r'):
#       modified_entry = {}

#       line_json = json.loads(line)
#       source = line_json["source_amrs"]
#       output = line_json["output_amr"]

#       # convert source and output amrs to Penman notation
#       new_source = linear_to_penman(source)
#       new_output = linear_to_penman(source)

#       modified_entry["source_amrs"] = new_source
#       modified_entry["output_amr"] = new_output
      
#       modified_entries.append(json.dumps(modified_entry))


#     with open(input_file + ".penman", 'w') as f:
#         for entry in modified_entries:
#             f.write("%s\n" % entry)


# # Convert linear AMR to Penman
# ######################
# # This is the function to use
# ######################
# def linear_to_penman(amr):
#     with open('/content/temp_converter/entry.jsonl', 'w') as f:
#       f.write(amr)

#     linear_to_penman_pipeline()

#     with open('/content/temp_converter/entry.jsonl.restored.txt', 'r') as f:
#       penman_amr = f.read().rstrip()

#     return penman_amr


# # Pipeline that calls the files to convert linear to AMR
# def linear_to_penman_pipeline():
#     !python char_level_AMR.py -f  /content/temp_converter/entry.jsonl  --super_chars
#     !python restoreAMR/restore_amr.py -f /content/temp_converter/entry.jsonl -o /content/temp_converter/entry.jsonl.restored
#     !python reformat_single_amrs.py -f /content/temp_converter/entry.jsonl.restored -v

In [ ]:
#@title
# # Converts linearized AMRs to Penman notation
# # Outdated function - converts a single AMR instead of a file full of AMRs
# import json

# # Loads linear AMRs from dataset and stores the Penman AMRs in new dataset
# def linear_to_penman_dataset(input_file):
  
#     modified_entries = []
#     for line in open(input_file, 'r'):
#       modified_entry = {}

#       amr = json.loads(line)

#       # convert source and output amrs to Penman notation
#       new_amr = linear_to_penman(amr)
      
#       modified_entries.append(new_amr)

#     output_file = input_file.replace(".json", "_penman.json")
#     with open(output_file, 'w') as f:
#         for entry in modified_entries:
#             f.write(json.dumps(entry) + "\n")


# # Convert linear AMR to Penman
# ######################
# # This is the function to use
# ######################
# def linear_to_penman(amr):
#     with open('/content/temp_converter/entry.jsonl', 'w') as f:
#       f.write(amr)

#     linear_to_penman_pipeline()

#     with open('/content/temp_converter/entry.jsonl.restored.txt', 'r') as f:
#       penman_amr = f.read().rstrip()

#     return penman_amr


# # Pipeline that calls the files to convert linear to AMR
# def linear_to_penman_pipeline():
#     !python char_level_AMR.py -f  /content/temp_converter/entry.jsonl  --super_chars 
#     !python restoreAMR/restore_amr.py -f /content/temp_converter/entry.jsonl -o /content/temp_converter/entry.jsonl.restored
#     !python reformat_single_amrs.py -f /content/temp_converter/entry.jsonl.restored -v

In [ ]:
from amr_utils import countparens


def countbrackets(amr):
  return amr.count("(") - amr.count(")")

infile = '/content/test_web/webnlg_gen.txt'

with open(infile, 'r') as f:
    lines = f.read()[:-1]
    amrs = lines.split("\n")  
            
    total_valid = 0
    total_amrs = 0

    with open(infile.replace(".txt", "_bracketized.txt"), 'w') as f_out:

        for i, amr in enumerate(amrs):
            brackets = countbrackets(amr)

            while brackets != 0:
              if brackets < 0:
                amr = amr[:-1]
              else:
                amr = amr + ')'
              brackets = countbrackets(amr)

            if countparens(amr):
              total_valid += 1
            else:
              print(amr)

            total_amrs += 1

            f_out.write(amr)
            if i != len(amrs)-1:
              f_out.write("\n")

        # print(valid_amr(amr))

    #         total_valid = 0
    #         total_amrs = 0
    #         for amr in amrs:
    #             is_valid = valid_amr(amr) 
    #             if(is_valid == True):
    #                 total_valid += 1
    #             total_amrs += 1
    print("Valid AMRs / Total # Output:")
    print(total_valid, "/", total_amrs)  
        # print(countparens(amr), brackets)

Valid AMRs / Total # Output:
1779 / 1779


In [ ]:
# Convert amrs conpied into the temp files below into their Penman notation
def to_penman():
  !python char_level_AMR.py -f  /content/test_web/test.jsonl  --super_chars --amr_ext ".jsonl"
  !python restoreAMR/restore_amr.py -f /content/test_web/test.char.jsonl -o /content/test_web/test.char.restored.json
  !python reformat_single_amrs.py -f /content/test_web/test.char.restored.json -v

In [ ]:
from google.colab import files
import re
# Convert amrs in the input file into the Penman notation
def convert_file_to_penman(inputfile):
  
  # Preprocess the AMRs
  with open(inputfile, 'r') as f:
    lines = f.read()#[:-1]
    amrs = lines.split("\n")
    amrs = [amr[1:-3] for amr in amrs]       # Remove the start and end quotes
    amrs = [re.sub('\\\\', '', amr) for amr in amrs]  # remove excess escape characters
  # Copy the processed input file AMRs into a temporary file
  with open('/content/test_web/test.jsonl', 'w') as f:
    f.write(amrs[0])
    for amr in amrs[1:]:
      f.write("\n" + amr)


  # Convert linearized AMRS in the temporary file to their Penman notation
  to_penman()


  # Copy converted AMRs from temporary files to intended output file
  with open('/content/test_web/test.char.restored.json.txt', 'r') as f:
    lines = f.read()

  outputfile = inputfile.replace(".json", "_penman.json")
  with open(outputfile, 'w') as f:
    f.write(lines)

  # Uncomment to download the output file
  # files.download(outputfile) 



In [ ]:
# Convert linear AMRs to Penman noation

# convert_file_to_penman('/content/test_web/test_repo.jsonl')   # test file
# convert_file_to_penman('/content/test_web/webnlg_test_target.json')   # webnlg_test
# convert_file_to_penman('/content/webnlg_test_source.json')   # webnlg_test
# convert_file_to_penman('/content/webnlg_test_target.json')   # webnlg_test

In [ ]:
# !ls ./test_web
# convert_file_to_penman('test_web/webnlg_test_source.json')   # webnlg_test
# convert_file_to_penman('test_web/webnlg_test_target.json')   # webnlg_test

In [ ]:
# # Uncomment to download the dev, test and train sets

# from google.colab import files
# files.download('/content/test_web/webnlg_test_target_penman.json') 

In [ ]:
#@title
# failed approach at brute amr concatenation

# for line in open("/content/test_web/test2.jsonl", 'r'):
#   line_json = json.loads(line)
#   source = line_json["source_amrs"]
#   amr_attr = []
#   headFound = False
#   for s in source:
#     # print(s, valid_amr(s))
#     s_split = s.split("\n")
#     if not headFound:
#       head = s_split[0]
#       headFound = True
#     attr = "\n".join(s_split[1:])
#     amr_attr.append(attr)
#     # print(attr)
#   # new_amr = head + "\n" + "\n".join(amr_attr)
#   new_amr = head + "\n" + amr_attr[0]  + "\n" +  amr_attr[0].replace("location", "ARG1").replace(":name", ":ARG2")
#   print(new_amr, valid_amr(new_amr))
#   print()
#   print("\n\n\n\n")

In [ ]:
# from amr_utils import valid_amr

# # count how many AMRs in Penman notation file are in valid format
# def check_penman_validity(inputfile):

#     with open(inputfile, 'r') as f:
#         lines = f.read()[:-1]
#         amrs = lines.split("\n\n")
#         total_valid = 0
#         total_amrs = 0
#         for amr in amrs:
#             is_valid = valid_amr(amr) 
#             if(is_valid == True):
#                 total_valid += 1
#             total_amrs += 1
#     print("Valid AMRs / Total # Output:")
#     print(total_valid, "/", total_amrs)  


In [ ]:
# convert_file_to_penman('/content/webnlg_gen')
# check_penman_validity('test_web/webnlg_test_target_penman.json')

In [ ]:
# from amr_utils import countparens
# import json

# def countparens_file(inputfile):
#     with open(inputfile) as file:
#         lines = file.read()
#         amrs = lines.split("\n")[:-1]
#         print(len(amrs))
        
#         valid_outputs = 0
#         total_outputs = 0
#         for amr in amrs:
          
#             amr = json.loads(amr)
#             amr = amr['output_amr']
#             if valid_amr(amr):
#                 # print(amr)   
#                 valid_outputs += 1
            
#             # if(countparens(amr)):
#             #     # print("Balanced parens example", valid_outputs)
#             #     # print(amr)
#             #     # print("\n")
#             #     valid_outputs += 1
#             else:
#                 print(amr)
#             #     pass
#             total_outputs += 1

#     print(f"{100*(valid_outputs / total_outputs):.2f}%, {total_outputs-valid_outputs}")
#     return




In [ ]:
from amr_utils import countparens
def countparens_file(inputfile):
    with open(inputfile) as file:
        lines = file.read()
        amrs = lines.split("\n")
        
        valid_outputs = 0
        total_outputs = 0
        for amr in amrs:
            if(countparens(amr)):
                # print("Balanced parens example", valid_outputs)
                # print(amr)
                # print("\n")
                valid_outputs += 1
            else:
                print(amr)
            total_outputs += 1

    print(f"{100*(valid_outputs / total_outputs):.2f}%")
    return




In [ ]:
# countparens_file('/content/test_web/webnlg_gen.txt')

In [ ]:
# from amr_utils import countparens
# valid_amr("(h0 / have-org-role-91\n      :ARG2 (c2 / capital)\n      :ARG0 (c1 / country\n            :name (n1 / name\n                  :op1 \"U.K.\" ))\n      :ARG1 (c0 / city\n            :name (n0 / name\n                  :op1 \"London\" )))")

In [ ]:
# countparens_file('/content/test_web/webnlg_train_parsed.jsonl')
# not sure why the script claims there are 4 unbalanced-paren amrs in the train set: I checked these by hand and they seem to be balanced

In [ ]:
countparens_file('/content/test_web/webnlg_dev_parsed.jsonl')

100.00%


In [ ]:
countparens_file('/content/test_web/webnlg_gen.txt.out')

FileNotFoundError: ignored